In [1]:
import numpy as np
import pandas as pd
import time
import datetime
import gc
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv('/root/data/train.csv')
test = pd.read_csv('/root/data/test.csv')
his = pd.read_csv('/root/data/historical_transactions.csv')
new = pd.read_csv('/root/data/new_merchant_transactions.csv')

In [3]:
his.head(3)

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37
1,Y,C_ID_4e6213e9bc,88,N,0,A,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16
2,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37


In [4]:
his['amount'] = (his.purchase_amount - his.purchase_amount.min())*1

In [ ]:
# 创建一个特征，用来计算每次交易后当次的还款日需要还款多少,我们简化一下，把最后还款日设定为下个月。
card_id_list = his.card_id.tolist()
amount_list = his.amount.tolist()
month_lag_list = his.month_lag.tolist()
installments_list = his.installments.tolist()


def repayment(card_id, amount, month_lag, installments, i):
    need_repay = 0
    i_min = i-6000
    i_max = i+6000
    
    if i_min < 0:
        i_min = 0
    if i_max > len(card_id):
        i_max = len(card_id)
        
    for n in range(i_min, i_max, 1):
        if card_id[n] == card_id[i]:
            if installments[n] == 0:
                if month_lag[n] == month_lag[i]:
                    need_repay += amount[n]
            else:
                if (month_lag[i] >= month_lag[n]+1) and (month_lag[i] <= month_lag[n]+installments[n]):
                    need_repay += amount[n]/installments[n]

    return need_repay
            
    
a = time.time()
need_pay = [repayment(card_id_list, amount_list, month_lag_list, installments_list, i) for i in range(len(card_id_list))]
print(time.time()-a)    

del card_id_list, amount_list, month_lag_list, installments_list
np.save('/root/tempfile/need_pay.npy', np.array(need_pay))

In [5]:
# 构造一个特征，用来计算其在每个月身上的债务总额。
# 我们假设每个月之内消费的款项，还款日在下一个月月底的最后一天。并且假设大家都在最后一天还款
# 首先，确定month_lag的最大值和最小值。
print(his.month_lag.max(), his.month_lag.min())

0 -13


In [10]:
his.reset_index(inplace=True)

In [ ]:
def debt(card_id, month_lag):
    global his, index_max, index_min
    for i in range(index_min, index_max+1):
        

In [ ]:
index_max = his.groupby('card_id').index.max()
index_min = his.groupby('card_id').index.min()

debt_columns = ['card_id'] + [i for i in np.arange(-13, 1, 1)]
debt_per_month = pd.DataFrame(np.zeros((index_max.shape[0], 15)), columns=debt_columns)

